![trainers in a store](trainers.jpg)

Sports clothing and athleisure attire is a huge industry, worth approximately [$193 billion in 2021](https://www.statista.com/statistics/254489/total-revenue-of-the-global-sports-apparel-market/) with a strong growth forecast over the next decade! 

In this notebook, you will undertake the role of a product analyst for an online sports clothing company. The company is specifically interested in how it can improve revenue. You will dive into product data such as pricing, reviews, descriptions, and ratings, as well as revenue and website traffic, to produce recommendations for its marketing and sales teams.  

You've been provided with four datasets to investigate:

#  brands.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `brand` | Brand of the product | 

# finance.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `listing_price` | Original price of the product | 
| `sale_price` | Discounted price of the product |
| `discount` | Discount off the listing price, as a decimal | 
| `revenue` | Revenue generated by the product |

# info.csv

| Columns | Description |
|---------|-------------|
| `product_name` | Name of the product | 
| `product_id` | Unique product identifier |
| `description` | Description of the product |

# reviews.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `rating` | Average product rating | 
| `reviews` | Number of reviews for the product |

In [128]:
import pandas as pd

brands = pd.read_csv("brands.csv") 
finance = pd.read_csv("finance.csv")
info = pd.read_csv("info.csv")
reviews = pd.read_csv("reviews.csv")

In [129]:
# Merge the data and drop null values
merged_df = info.merge(finance, on="product_id")
merged_df = merged_df.merge(reviews, on="product_id")
merged_df = merged_df.merge(brands, on="product_id")
merged_df.dropna(inplace=True)

### Problem 1

* What is the volume of products and average revenue for Adidas and Nike products based on listing price quartiles?
    * Label products priced up to quartile one as `"Budget"`, quartile two as `"Average"`, quartile three as `"Expensive"`, and quartile four as `"Elite"`.
    * Store as a `pandas` DataFrame called adidas_vs_nike containing the following columns: `"brand"`, `"price_label"`, `"num_products"`, and `"mean_revenue"`. All numeric values should be rounded to two decimal places.

In [130]:
# Group listing prices into quartiles
merged_df['price_label'] = pd.qcut(finance['listing_price'], 4,
                                   labels=['Budget', 'Average', 'Expensive', 'Elite'])

# By price quartile & brand, compute the volume of products & average revenue 
adidas_vs_nike = merged_df.groupby(['brand', 'price_label'], as_index=False)\
                          .agg(num_products=('price_label', 'count'),
                               mean_revenue=('revenue', 'mean'))\
                          .round(2)

adidas_vs_nike

,brand,price_label,num_products,mean_revenue
0,Adidas,Budget,574,2015.68
1,Adidas,Average,655,3035.30
2,Adidas,Expensive,759,4621.56
3,Adidas,Elite,587,8302.78
4,Nike,Budget,357,1596.33
5,Nike,Average,8,675.59
6,Nike,Expensive,47,500.56
7,Nike,Elite,130,1367.45


### Problem 2

* Do any differences exist between the word count of a product's description and its mean rating?
    * Split product description length into bins of 100 characters and calculate the average rating and number of reviews.
    * Store the results as a `pandas` DataFrame called `description_lengths` containing the following columns: `"description_length"`, `"mean_rating"`, `"num_reviews"`, again rounding numeric values to two decimal places.

In [131]:
# Group description lengths by 100s of characters - max description length < 700
merged_df['description_length'] = pd.cut(merged_df['description'].str.len(),
                                         bins=list(range(0, 800, 100)), 
                                         labels=[str(x) for x in range(100, 800, 100)])

# By description length, compute the average ratings & number of reviews
description_lengths = merged_df.groupby(['description_length'], as_index=False)\
                               .agg(mean_rating=('rating', 'mean'),
                                    num_reviews=('reviews', 'count'))\
                               .round(2)

description_lengths

,description_length,mean_rating,num_reviews
0,100,2.26,7
1,200,3.19,526
2,300,3.28,1785
3,400,3.29,651
4,500,3.35,118
5,600,3.12,15
6,700,3.65,15
